## RAG using DSPy & OpenAI

In this notebook, we'll build a full RAG framework using the trained and indexed retrieval model

In [1]:
# importing packages
import json
import dspy
import os
from dotenv import load_dotenv
from dspy.datasets import HotPotQA
from dspy.teleprompt import BootstrapFewShot
from ragatouille import RAGPretrainedModel
from dspy.evaluate.evaluate import Evaluate
from dsp.utils import deduplicate
from rich import print

load_dotenv()

No CUDA runtime is found, using CUDA_HOME='/usr'


True

**RAG**

In [15]:
# configuring dspy

openai_api_key = os.environ.get("OPENAI_API_KEY")
turbo = turbo = dspy.OpenAI(model='gpt-3.5-turbo', api_key=openai_api_key)
dspy.settings.configure(lm=turbo)

In [4]:
# loading the dataset we'll use to train our RAG system

dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

/home/ubuntu/miniconda3/envs/dspy/lib/python3.9/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


**Basic RAG**

In [5]:
# function to load the trained retrieval model
def query_index(query: str, topk: int):

    # provide the path to your index
    index_path = ".ragatouille/colbert/indexes/EPL"
    try:
        RAG = RAGPretrainedModel.from_index(index_path=index_path)
        return RAG.search(query=query, k=topk)
    except FileNotFoundError as e:
        print(f"Index not found: {e}") 

In [6]:
# Signature
class GenerateAnswer(dspy.Signature): 
    """Answer questions with short factoid answers."""
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 100 words")
    
# Module / Pipeline
class RAG(dspy.Module): 
    def __init__(self, num_passages=3):
        super().__init__()
        #self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.topk = num_passages
    
    def forward(self, question):
        #context = self.retrieve(question).passages
        context_dict = query_index(query=question, topk=self.topk)
        context = [x['content'] for x in context_dict]
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

# Optimizer / Optimising Pipeline
def validate_context_and_answer(example, pred, trace=None): 
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

In [7]:
teleprompter = BootstrapFewShot(metric=validate_context_and_answer) 
compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

  0%|          | 0/20 [00:00<?, ?it/s]/home/ubuntu/miniconda3/envs/dspy/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[May 13, 02:13:25] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/home/ubuntu/miniconda3/envs/dspy/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ubuntu/miniconda3/envs/dspy/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


Loading searcher for index EPL for the first time... This may take a few seconds
[May 13, 02:13:28] #> Loading codec...
[May 13, 02:13:28] #> Loading IVF...
[May 13, 02:13:28] Loading segmented_lookup_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[May 13, 02:13:28] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 1049.36it/s]

...

100%|██████████| 1/1 [00:00<00:00, 900.26it/s]

[May 13, 02:16:20] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 70.83it/s]

Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . This American guitarist best known for her work with the Iron Maidens is an ancestor of a composer who was known as what?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2023,  2137,  5990,  2190,  2124,  2005,  2014,  2147,
         2007,  1996,  3707, 10494,  2015,  2003,  2019, 13032,  1997,  1037,
         4543,  2040,  2001,  2124,  2004,  2054,  1029,   102,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0])




100%|██████████| 20/20 [03:02<00:00,  9.10s/it]


In [8]:
query="Which club has the record for the longest unbeaten run in the premier league?"
pred = compiled_rag(query)

Loading searcher for index EPL for the first time... This may take a few seconds
[May 13, 02:16:57] #> Loading codec...
[May 13, 02:16:57] #> Loading IVF...
[May 13, 02:16:57] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 1839.61it/s]

[May 13, 02:16:57] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 70.51it/s]

Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Which club has the record for the longest unbeaten run in the premier league?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2029,  2252,  2038,  1996,  2501,  2005,  1996,  6493,
        20458,  2448,  1999,  1996,  4239,  2223,  1029,   102,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



In [9]:
print(f"Question: {query}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Question: Which club has the record for the longest unbeaten run in the premier league?

Predicted Answer: Arsenal

Retrieved Contexts (truncated): ['Arsenal hold the record for the longest run of unbeaten League matches (49 
between May 2003 and October 2004). This included all 38 matches of their title-winning 2003–04 season, when 
Arsenal became o...', 'Herbert Chapman won the club its first silverware, and his legacy enabled a trophy-laden 
period in the 1930s. He helped introduce the WM formation, floodlights, and shirt numbers; he also added the 
wh...', 'Chelsea hold the English record for the fewest goals conceded during a league season (15), the highest 
number of clean sheets overall in a Premier League season (25) (both set during the 2004–05 seaso...']

###

**Uncompile Baleen without Optimizer**

In [10]:
def gold_passages_retrieved(example, pred, trace=None):
    gold_titles = set(map(dspy.evaluate.normalize_text, example['gold_titles']))
    found_titles = set(map(dspy.evaluate.normalize_text, [c.split(' | ')[0] for c in pred.context]))
    return gold_titles.issubset(found_titles)

evaluate_on_hotpotqa = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)
compiled_rag_retrieval_score = evaluate_on_hotpotqa(compiled_rag, metric=gold_passages_retrieved)

# 5b. Uncompiled Baleen RAG (without Optimizer)
class GenerateSearchQuery(dspy.Signature):
    """Write a simple search query that will help answer a complex question."""
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()

class SimplifiedBaleen(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()
        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops
    
    def forward(self, question):
        context = []
        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query
            #passages = self.retrieve(query).passages
            context_dict = query_index(query=query, topk=3)
            passages = [x['content'] for x in context_dict]
            context = deduplicate(context + passages)
        pred = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=pred.answer)

  0%|          | 0/50 [00:00<?, ?it/s]Loading searcher for index EPL for the first time... This may take a few seconds
[May 13, 02:18:50] #> Loading codec...
[May 13, 02:18:50] #> Loading IVF...
[May 13, 02:18:50] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 966.88it/s]

[May 13, 02:18:50] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 71.76it/s]

Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Are both Cangzhou and Qionghai in the Hebei province of China?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2024,  2119,  2064,  2290,  9791,  1998, 18816,  5063,
        10932,  1999,  1996,  2002, 19205,  2874,  1997,  2859,  1029,   102,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



...

[May 13, 02:26:36] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 64.62it/s]

Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . In what city was the Election Law Journal founded?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([ 101,    1, 1999, 2054, 2103, 2001, 1996, 2602, 2375, 3485, 2631, 1029,
         102,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,
         103,  103,  103,  103,  103,  103,  103,  103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



Average Metric: 0 / 50  (0.0): 100%|██████████| 50/50 [07:54<00:00,  9.49s/it]


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","[""Shenzhen Peng City (2019–present)\nOn 20 February 2019, it was announced that CFG as well as UBTECH and China Sports Capital had acquired Sichuan Jiuniu F.C.\n...","No. Cangzhou is in Hebei province, while Qionghai is in Hainan province.",False
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017 NHL Expansion Draft', '2017–18 Pittsburgh Penguins season'}","[""It restored its sponsorship after the change of ownership of the club.\nFollowing the introduction of sleeve sponsors in the Premier League, Chelsea had Alliance Tyres...",National Hockey League (NHL),False
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","[""In his 21st and final season, Arsenal under Arsene Wenger finished sixth and won the community shield. Wenger departed Arsenal following the end of the...",Steve Yzerman,False
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","[""A new training ground was constructed at the site for £45 million, which opened in 2012. The 77-acre site has 15 grass pitches and one-and-a-half...",River Tyne,False
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","['== Crest ==\n\nSince the 1921 FA Cup Final the Tottenham Hotspur crest has featured a cockerel. Harry Hotspur, after whom the club is named, was...",Alfred the Great,False


In [11]:
uncompiled_baleen = SimplifiedBaleen()  # uncompiled (i.e., zero-shot) program
pred = uncompiled_baleen(query)

/home/ubuntu/miniconda3/envs/dspy/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ubuntu/miniconda3/envs/dspy/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


Loading searcher for index EPL for the first time... This may take a few seconds
[May 13, 02:27:53] #> Loading codec...
[May 13, 02:27:53] #> Loading IVF...
[May 13, 02:27:53] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 1462.96it/s]

[May 13, 02:27:53] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 36.93it/s]

...

Loading searcher for index EPL for the first time... This may take a few seconds
[May 13, 02:28:02] #> Loading codec...
[May 13, 02:28:02] #> Loading IVF...
[May 13, 02:28:02] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 1596.01it/s]

[May 13, 02:28:02] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 55.93it/s]

Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . "Which football club holds the record for the longest unbeaten run in the Premier League?", 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  1000,  2029,  2374,  2252,  4324,  1996,  2501,  2005,
         1996,  6493, 20458,  2448,  1999,  1996,  4239,  2223,  1029,  1000,
          102,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



In [12]:
print(f"Question: {query}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Question: Which club has the record for the longest unbeaten run in the premier league?

Predicted Answer: Arsenal

Retrieved Contexts (truncated): ['Arsenal hold the record for the longest run of unbeaten League matches (49 
between May 2003 and October 2004). This included all 38 matches of their title-winning 2003–04 season, when 
Arsenal became o...', 'Herbert Chapman won the club its first silverware, and his legacy enabled a trophy-laden 
period in the 1930s. He helped introduce the WM formation, floodlights, and shirt numbers; he also added the 
wh...', 'Chelsea hold the English record for the fewest goals conceded during a league season (15), the highest 
number of clean sheets overall in a Premier League season (25) (both set during the 2004–05 seaso...']

**Compiled Baleen RAG with Optimizer**

In [13]:
# 5c. Compiled Baleen RAG (with Optimizer)
def validate_context_and_answer_and_hops(example, pred, trace=None):
    if not dspy.evaluate.answer_exact_match(example, pred): return False
    if not dspy.evaluate.answer_passage_match(example, pred): return False
    hops = [example.question] + [outputs.query for *_, outputs in trace if 'query' in outputs]
    if max([len(h) for h in hops]) > 100: return False
    if any(dspy.evaluate.answer_exact_match_str(hops[idx], hops[:idx], frac=0.8) for idx in range(2, len(hops))): return False
    return True

In [16]:
teleprompter = BootstrapFewShot(metric=validate_context_and_answer_and_hops)
compiled_baleen = teleprompter.compile(SimplifiedBaleen(), teacher=SimplifiedBaleen(passages_per_hop=2), trainset=trainset)
uncompiled_baleen_retrieval_score = evaluate_on_hotpotqa(uncompiled_baleen, metric=gold_passages_retrieved)
compiled_baleen_retrieval_score = evaluate_on_hotpotqa(compiled_baleen, metric=gold_passages_retrieved)

print(f"## Retrieval Score for RAG: {compiled_rag_retrieval_score}")
print(f"## Retrieval Score for uncompiled Baleen: {uncompiled_baleen_retrieval_score}")
print(f"## Retrieval Score for compiled Baleen: {compiled_baleen_retrieval_score}")

/home/ubuntu/miniconda3/envs/dspy/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ubuntu/miniconda3/envs/dspy/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


Loading searcher for index EPL for the first time... This may take a few seconds
[May 13, 02:39:19] #> Loading codec...
[May 13, 02:39:19] #> Loading IVF...
[May 13, 02:39:19] #> Loading doclens...



100%|██████████| 1/1 [00:00<00:00, 949.58it/s]

[May 13, 02:39:19] #> Loading codes and residuals...




100%|██████████| 1/1 [00:00<00:00, 53.72it/s]

...

Average Metric: 0 / 50  (0.0): 100%|██████████| 50/50 [05:40<00:00,  6.82s/it]


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","[""Shenzhen Peng City (2019–present)\nOn 20 February 2019, it was announced that CFG as well as UBTECH and China Sports Capital had acquired Sichuan Jiuniu F.C.\n...",No,False
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017 NHL Expansion Draft', '2017–18 Pittsburgh Penguins season'}",['and Goal of the Season. The Young Player of the Season award is given to the most outstanding U-23 player starting from the 2019–20 season.\nThe...,NHL,False
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}",['Fourteen Liverpool fans received convictions for involuntary manslaughter.\nFagan had announced his retirement just before the disaster and Kenny Dalglish was appointed as player-manager. During his...,Steve Yzerman,False
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Castle', 'Crichton Collegiate Church'}","[""A new training ground was constructed at the site for £45 million, which opened in 2012. The 77-acre site has 15 grass pitches and one-and-a-half...",Irwell,False
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","[""Promoted to the board a few days after the Munich air disaster, Louis Edwards, a friend of Matt Busby, began acquiring shares in the club;...",Alfred the Great,False


## Retrieval Score for RAG: 0.0

## Retrieval Score for uncompiled Baleen: 0.0

## Retrieval Score for compiled Baleen: 0.0

In [51]:
query = "Who is the most successful manager in the premier league and how many titles did he win?"
pred = compiled_baleen(query)
print(f"Question: {query}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

/home/ubuntu/miniconda3/envs/dspy/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ubuntu/miniconda3/envs/dspy/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


Loading searcher for index EPL for the first time... This may take a few seconds
[May 13, 03:35:00] #> Loading codec...
[May 13, 03:35:00] #> Loading IVF...
[May 13, 03:35:00] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 2759.41it/s]

[May 13, 03:35:00] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 100.27it/s]

...


100%|██████████| 1/1 [00:00<00:00, 99.19it/s]

Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Most successful manager in Premier League titles won., 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([ 101,    1, 2087, 3144, 3208, 1999, 4239, 2223, 4486, 2180, 1012,  102,
         103,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,
         103,  103,  103,  103,  103,  103,  103,  103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



Question: Who is the most successful manager in the premier league and how many titles did he win?

Predicted Answer: Sir Alex Ferguson won 13 league titles.

Retrieved Contexts (truncated): ['Arsène Wenger is the longest-serving manager, having been in charge of Arsenal in
the Premier League from 1996 to his departure at the conclusion of the 2017–18 season, and holds the record for 
most m...', "Sir Alex Ferguson is the club's longest-serving and most successful manager, winning 38 trophies, 
including 13 league titles, five FA Cups, and two Champions League titles between 1986 and 2013. In th...", 
'Herbert Chapman won the club its first silverware, and his legacy enabled a trophy-laden period in the 1930s. He 
helped introduce the WM formation, floodlights, and shirt numbers; he also added the wh...', "Pep Guardiola, former
head coach of Barcelona and Bayern Munich, was confirmed to become Manchester City's new manager on 1 February 
2016, with the announcement having been made several months before ..."]

In [52]:
history = turbo.inspect_history(n=1)




Answer questions with short factoid answers.

---

Question: "Everything Has Changed" is a song from an album released under which record label ?
Answer: Big Machine Records

Question: On the coast of what ocean is the birthplace of Diogal Sakho?
Answer: Atlantic

Question: Samantha Cristoforetti and Mark Shuttleworth are both best known for being first in their field to go where?
Answer: space

Question: Which company distributed this 1977 American animated film produced by Walt Disney Productions for which Sherman Brothers wrote songs?
Answer: Buena Vista Distribution

Question: Which Pakistani cricket umpire who won 3 consecutive ICC umpire of the year awards in 2009, 2010, and 2011 will be in the ICC World Twenty20?
Answer: Aleem Sarwar Dar

Question: Which American actress who made their film debut in the 1995 teen drama "Kids" was the co-founder of Voto Latino?
Answer: Rosario Dawson

Question: This American guitarist best known for her work with the Iron Maidens is an ancesto

In [72]:
print(f"Contex & Reasoning:\n\n{history[-3358:]}")

Contex & Reasoning:

[1] «Arsène Wenger is the longest-serving manager, having been in charge of Arsenal in the Premier League from 1996
to his departure at the conclusion of the 2017–18 season, and holds the record for most matches managed in the 
Premier League with 828, all with Arsenal. He broke the record set by Alex Ferguson, who had managed 810 matches 
with Manchester United from the Premier League's inception to his retirement at the end of the 2012–13 season. 
Ferguson was in charge of Manchester United from November 1986 until his retirement at the end of the 2012–13 
season, meaning he was manager for the last five years of the old Football League First Division and all of the 
first 21 seasons of the Premier League.
Notably, since its creation the Premier League has never been won by an English manager.»
[2] «Sir Alex Ferguson is the club's longest-serving and most successful manager, winning 38 trophies, including 13
league titles, five FA Cups, and two Champions League titles between 1986 and 2013. In the 1998–99 season, under 
Ferguson, the club became the first in the history of English football to achieve the continental treble of the 
Premier League, FA Cup and UEFA Champions League. In winning the UEFA Europa League under José Mourinho in 2016–17,
they became one of five clubs to have won the original three main UEFA club competitions (the Champions League, 
Europa League and Cup Winners' Cup).
Manchester United is one of the most widely supported football clubs in the world and has rivalries with Liverpool,
Manchester City, Arsenal and Leeds United.»
[3] «Herbert Chapman won the club its first silverware, and his legacy enabled a trophy-laden period in the 1930s. 
He helped introduce the WM formation, floodlights, and shirt numbers; he also added the white sleeves and brighter 
red to the club's jersey. Arsène Wenger is the club's longest-serving manager and has won the most trophies. He won
a record seven FA Cups, and his title-winning team set an English record for the longest top-flight unbeaten league
run at 49 games between 2003 and 2004, receiving the nickname The Invincibles.
In 2006, the club moved to the nearby Emirates Stadium.»
[4] «Pep Guardiola, former head coach of Barcelona and Bayern Munich, was confirmed to become Manchester City's new
manager on 1 February 2016, with the announcement having been made several months before Manuel Pellegrini left his
position. Guardiola's first season in Manchester would end trophyless, with the Blues placing third in the league 
standings, but the following season proved far more successful, as City won the Premier League title with the 
highest points total in history and broke numerous other club and English league records along the way.
This would prove to be the start of a period of unprecedented success for Manchester City under Guardiola. Between 
the 2017–18 and 2022–23 Premier League seasons, City won five out of possible six league titles, only finishing 
second behind Liverpool in the 2019–20 season.»

Question: Who is the most successful manager in the premier league and how many titles did he win?

Reasoning: Let's think step by step in order to[32m produce the answer. We know from the context that Sir Alex 
Ferguson is the most successful manager in the Premier League. He won 13 league titles.

Answer: Sir Alex Ferguson won 13 league titles.[0m